## Twitter API v1.1 Client

In [2]:
import tweepy
import pandas as pd 
import numpy as np

## Autenticação 

In [3]:
consumer_key= 'drhztkGNhICDt3cwBGXi7zzQk'
consumer_secret= 'FDeCBlSi4qIbR9oPsf3tphQjCO6HqZfYANWceYIdJDZHsOaabi'
access_token= '207778119-E9W1kqRXBbNeU2n8vcA0OoMn9XZpE9yvmZrD8gUx'
access_token_secret= 'dncbYQQrh5jlJ97OmgxeaGgqOFiC7jnNpXdPngWMCRtDE'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


## Query API V1

In [4]:
# Define the search term and the date_since date as variables
search_words = "DisneyPlus"
date_since = "2020-11-03"
lang = 'pt'

tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang=lang,
              since=date_since).items(50)

## Resultados: Cursor to List

In [5]:
todos = list(tweets)

## To DF and CSV File

In [25]:
def tweets_to_dataframe(lista):

    columns = set()
    
    tweets_data = []

    
    for tweet in todos:
        keys = tweet._json.keys()
        single_data = {}
        
        
        for k in keys:
            try:
                single_data[k] = tweet._json.get(k)
                    
            except Exception as e:
                print(e)
                single_data[k] = np.nan
                
            columns.add(k)
        tweets_data.append(single_data)    
    

    header_cols = list(columns)
    
    df = pd.DataFrame(tweets_data, columns=header_cols)
    
    
    return df

# Consertando as colunas

## Destrinchando coluna Entities - Descobrindo quem o usuario mencionou

In [48]:
def mentions(df):

        df['screen_name_mentions_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['screen_name'] if x['user_mentions'] else np.nan)

        df['id_mentions_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['id_str'] if x['user_mentions'] else np.nan)

        return df


## Destrinchando coluna Retweeted_Status - Descobrindo de quem o usuario retweetou

In [51]:
def retweets(df):

    df['retweeted_from_screen_name'] = df['retweeted_status'].apply(lambda x: x['user']['screen_name'] if x is not np.nan else np.nan)

    df['retweeted_from_id'] = df['retweeted_status'].apply(lambda x: x['user']['id_str'] if x is not np.nan else np.nan)

    df['retweeted_from_is_verified'] = df['retweeted_status'].apply(lambda x: x['user']['verified'] if x is not np.nan else np.nan)

    return df


## Destrinchando coluna User - Descobrindo quem e o usuario

In [52]:
def user(df):

    df['user_screen_name'] = df['user'].apply(lambda x: x['screen_name'] if x is not np.nan else np.nan)

    df['user_id'] = df['user'].apply(lambda x: x['id'] if x is not np.nan else np.nan)

    df['user_followers'] = df['user'].apply(lambda x: x['followers_count'] if x is not np.nan else np.nan)

    df['user_following'] = df['user'].apply(lambda x: x['friends_count'] if x is not np.nan else np.nan)

    df['user_is_verified'] = df['user'].apply(lambda x: x['verified'] if x is not np.nan else np.nan)

    return df

In [10]:
def destrinchando_colunas(df):
    
    mentions(df)
    retweets(df)
    user(df)

    return df

## Gerando CSV File

In [56]:
def create_csv(df):
    
    df.to_csv(r'C:\Users\amand\Documents\Jupyter Notebooks\DisneyPlus\Data\tweets.csv')

## Aplicando as funcoes

In [26]:
df = tweets_to_dataframe(todos)

In [53]:
df2 = destrinchando_colunas(df)

In [55]:
df2.head()

,retweeted,truncated,in_reply_to_user_id_str,place,text,extended_entities,source,in_reply_to_status_id,favorite_count,possibly_sensitive,...,screen_name_mentions_1,id_mentions_1,retweeted_from_screen_name,retweeted_from_id,retweeted_from_is_verified,user_screen_name,user_id,user_followers,user_following,user_is_verified
0,False,False,None,None,RT @DisneyPlusBR: Prepare-se para uma missão d...,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,0,NaN,...,DisneyPlusBR,1011372006753746944,DisneyPlusBR,1011372006753746944,True,616CAPTMARVEL,366871108,1622,4741,False
1,False,False,None,None,RT @Binuel_Isulio: Tá tudo perfeito ( tirando ...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,0,NaN,...,Binuel_Isulio,1142169627792986112,Binuel_Isulio,1142169627792986112,False,ilysmbinuel,1236073778033184772,365,167,False
2,False,False,None,None,RT @disneybrasil: #DisneyPlus: O ator brasilei...,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,0,NaN,...,disneybrasil,41924663,disneybrasil,41924663,False,moonthatlight,342959597,351,1635,False
3,False,False,None,None,A Brie Larson é perspicaz em tudo que faz. Até...,"{'media': [{'id': 1325633049627799554, 'id_str...","<a href=""http://twitter.com/download/iphone"" r...",NaN,5,False,...,NaN,NaN,NaN,NaN,NaN,juslxrson,70022571,1418,1363,False
4,False,False,None,None,"RT @WoompBR: Jorge Blanco canta a música ""Vivo...",NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,0,NaN,...,WoompBR,977308058,WoompBR,977308058,False,WoompBR,977308058,24571,878,False


In [57]:
create_csv(df2)

## Cleanning unecessary columns

In [4]:
df = pd.read_csv('Data/tweets.csv')

In [22]:
pd.set_option("display.max_columns", 50)
df

,Unnamed: 0,retweeted,truncated,in_reply_to_user_id_str,place,text,extended_entities,source,in_reply_to_status_id,favorite_count,possibly_sensitive,coordinates,contributors,lang,created_at,favorited,retweeted_status,in_reply_to_status_id_str,metadata,entities,retweet_count,in_reply_to_screen_name,quoted_status_id,id,is_quote_status,in_reply_to_user_id,quoted_status_id_str,user,id_str,quoted_status,geo,screen_name_mentions_1,id_mentions_1,retweeted_from_screen_name,retweeted_from_id,retweeted_from_is_verified,user_screen_name,user_id,user_followers,user_following,user_is_verified
0,0,False,False,NaN,NaN,RT @DisneyPlusBR: Prepare-se para uma missão d...,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,0,NaN,NaN,NaN,pt,Wed Nov 11 02:23:53 +0000 2020,False,{'created_at': 'Mon Nov 09 22:30:19 +0000 2020...,NaN,"{'iso_language_code': 'pt', 'result_type': 're...",{'hashtags': [{'text': 'CandaceContraOUniverso...,66,NaN,NaN,1326349902180470784,False,NaN,NaN,"{'id': 366871108, 'id_str': '366871108', 'name...",1326349902180470784,NaN,NaN,DisneyPlusBR,1.011372e+18,DisneyPlusBR,1.011372e+18,True,616CAPTMARVEL,366871108,1622,4741,False
1,1,False,False,NaN,NaN,RT @Binuel_Isulio: Tá tudo perfeito ( tirando ...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,0,NaN,NaN,NaN,pt,Wed Nov 11 02:21:08 +0000 2020,False,{'created_at': 'Wed Nov 11 02:14:18 +0000 2020...,NaN,"{'iso_language_code': 'pt', 'result_type': 're...","{'hashtags': [], 'symbols': [], 'user_mentions...",2,NaN,1.326316e+18,1326349207167508481,True,NaN,1.326316e+18,"{'id': 1236073778033184772, 'id_str': '1236073...",1326349207167508481,NaN,NaN,Binuel_Isulio,1.142170e+18,Binuel_Isulio,1.142170e+18,False,ilysmbinuel,1236073778033184772,365,167,False
2,2,False,False,NaN,NaN,RT @disneybrasil: #DisneyPlus: O ator brasilei...,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,0,NaN,NaN,NaN,pt,Wed Nov 11 02:20:47 +0000 2020,False,{'created_at': 'Wed Nov 11 02:17:00 +0000 2020...,NaN,"{'iso_language_code': 'pt', 'result_type': 're...","{'hashtags': [{'text': 'DisneyPlus', 'indices'...",1,NaN,NaN,1326349122580918275,False,NaN,NaN,"{'id': 342959597, 'id_str': '342959597', 'name...",1326349122580918275,NaN,NaN,disneybrasil,4.192466e+07,disneybrasil,4.192466e+07,False,moonthatlight,342959597,351,1635,False
3,3,False,False,NaN,NaN,A Brie Larson é perspicaz em tudo que faz. Até...,"{'media': [{'id': 1325633049627799554, 'id_str...","<a href=""http://twitter.com/download/iphone"" r...",NaN,5,False,NaN,NaN,pt,Wed Nov 11 02:20:08 +0000 2020,False,NaN,NaN,"{'iso_language_code': 'pt', 'result_type': 're...","{'hashtags': [], 'symbols': [], 'user_mentions...",0,NaN,NaN,1326348957384122368,False,NaN,NaN,"{'id': 70022571, 'id_str': '70022571', 'name':...",1326348957384122368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,juslxrson,70022571,1418,1363,False
4,4,False,False,NaN,NaN,"RT @WoompBR: Jorge Blanco canta a música ""Vivo...",NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,0,NaN,NaN,NaN,pt,Wed Nov 11 02:19:59 +0000 2020,False,{'created_at': 'Wed Nov 11 02:08:23 +0000 2020...,NaN,"{'iso_language_code': 'pt', 'result_type': 're...","{'hashtags': [], 'symbols': [], 'user_mentions...",6,NaN,NaN,1326348921535389698,False,NaN,NaN,"{'id': 977308058, 'id_str': '977308058', 'name...",1326348921535389698,NaN,NaN,WoompBR,9.773081e+08,WoompBR,9.773081e+08,False,WoompBR,977308058,24571,878,False
5,5,False,False,NaN,NaN,"RT @DisneyPlusBR: E aí, com qual personagem vo...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,0,False,NaN,NaN,pt,Wed Nov 11 02:18:39 +0000 2020,False,{'created_at': 'Mon Nov 09 21:04:30 +0000 2020...,NaN,"{'iso_language_code': 'pt', 'result_type': 're...","{'hashtags': [], 'symbols': [], 'user_mentions...",134,NaN,NaN,1326348585265410048,False,NaN,NaN,"{'id': 60151856, 'id_str': '60151856', 'name':...",1326348585265410048,NaN,NaN,DisneyPlusBR,1.011372e+18,DisneyPlusBR,1.011372e+18,True,hoyosedu,60151856,1090,497,False
6,6,False,False,NaN,NaN,"

In [7]:
df.columns

Index(['Unnamed: 0', 'retweeted', 'truncated', 'in_reply_to_user_id_str',
       'place', 'text', 'extended_entities', 'source', 'in_reply_to_status_id',
       'favorite_count', 'possibly_sensitive', 'coordinates', 'contributors',
       'lang', 'created_at', 'favorited', 'retweeted_status',
       'in_reply_to_status_id_str', 'metadata', 'entities', 'retweet_count',
       'in_reply_to_screen_name', 'quoted_status_id', 'id', 'is_quote_status',
       'in_reply_to_user_id', 'quoted_status_id_str', 'user', 'id_str',
       'quoted_status', 'geo', 'screen_name_mentions_1', 'id_mentions_1',
       'retweeted_from_screen_name', 'retweeted_from_id',
       'retweeted_from_is_verified', 'user_screen_name', 'user_id',
       'user_followers', 'user_following', 'user_is_verified'],
      dtype='object')

In [12]:
all_columns = list(df.columns)

In [14]:
all_columns

['Unnamed: 0',
 'retweeted',
 'truncated',
 'in_reply_to_user_id_str',
 'place',
 'text',
 'extended_entities',
 'source',
 'in_reply_to_status_id',
 'favorite_count',
 'possibly_sensitive',
 'coordinates',
 'contributors',
 'lang',
 'created_at',
 'favorited',
 'retweeted_status',
 'in_reply_to_status_id_str',
 'metadata',
 'entities',
 'retweet_count',
 'in_reply_to_screen_name',
 'quoted_status_id',
 'id',
 'is_quote_status',
 'in_reply_to_user_id',
 'quoted_status_id_str',
 'user',
 'id_str',
 'quoted_status',
 'geo',
 'screen_name_mentions_1',
 'id_mentions_1',
 'retweeted_from_screen_name',
 'retweeted_from_id',
 'retweeted_from_is_verified',
 'user_screen_name',
 'user_id',
 'user_followers',
 'user_following',
 'user_is_verified']

In [23]:
unecessary_columns = ['Unnamed: 0', 'truncated', 'possibly_sensitive', 'metadata', 'quoted_status', 'quoted_status_id_str']

In [24]:
graph_columns = ['user_screen_name', 'user_id', 'user_is_verified', 'retweeted_from_screen_name', 'retweeted_from_id', 'retweeted_from_is_verified', 'screen_name_mentions_1' , 'id_mentions_1', 'in_reply_to_screen_name', 'in_reply_to_user_id', 'created_at', 'id', 'text']

In [25]:
df_graph = df[graph_columns]

In [26]:
df_graph

,user_screen_name,user_id,user_is_verified,retweeted_from_screen_name,retweeted_from_id,retweeted_from_is_verified,screen_name_mentions_1,id_mentions_1,in_reply_to_screen_name,in_reply_to_user_id,created_at,id,text
0,616CAPTMARVEL,366871108,False,DisneyPlusBR,1.011372e+18,True,DisneyPlusBR,1.011372e+18,NaN,NaN,Wed Nov 11 02:23:53 +0000 2020,1326349902180470784,RT @DisneyPlusBR: Prepare-se para uma missão d...
1,ilysmbinuel,1236073778033184772,False,Binuel_Isulio,1.142170e+18,False,Binuel_Isulio,1.142170e+18,NaN,NaN,Wed Nov 11 02:21:08 +0000 2020,1326349207167508481,RT @Binuel_Isulio: Tá tudo perfeito ( tirando ...
2,moonthatlight,342959597,False,disneybrasil,4.192466e+07,False,disneybrasil,4.192466e+07,NaN,NaN,Wed Nov 11 02:20:47 +0000 2020,1326349122580918275,RT @disneybrasil: #DisneyPlus: O ator brasilei...
3,juslxrson,70022571,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wed Nov 11 02:20:08 +0000 2020,1326348957384122368,A Brie Larson é perspicaz em tudo que faz. Até...
4,WoompBR,977308058,False,WoompBR,9.773081e+08,False,WoompBR,9.773081e+08,NaN,NaN,Wed Nov 11 02:19:59 +0000 2020,1326348921535389698,"RT @WoompBR: Jorge Blanco canta a música ""Vivo..."
5,hoyosedu,60151856,False,DisneyPlusBR,1.011372e+18,True,DisneyPlusBR,1.011372e+18,NaN,NaN,Wed Nov 11 02:18:39 +0000 2020,1326348585265410048,"RT @DisneyPlusBR: E aí, com qual personagem vo..."
6,JosyMendesp,205984208,False,DisneyPlusBR,1.011372e+18,True,DisneyPlusBR,1.011372e+18,NaN,NaN,Wed Nov 11 02:17:36 +0000 2020,1326348319292092418,"RT @DisneyPlusBR: E aí, com qual personagem vo..."
7,iambrurc,563875903,False,WoompBR,9.773081e+08,False,WoompBR,9.773081e+08,NaN,NaN,Wed Nov 11 02:17:21 +0000 2020,1326348258025873409,"RT @WoompBR: Jorge Blanco canta a música ""Vivo..."
8,Rayf_Bergann,333140636,False,DisneyPlusBR,1.011372e+18,True,DisneyPlusBR,1.011372e+18,NaN,NaN,Wed Nov 11 02:17:02 +0000 2020,1326348178417922048,RT @DisneyPlusBR: Confira a nova capa da @EW d...
9,disneybrasil,41924663,False,NaN,NaN,NaN,natgeobrasil,2.858982e+07,NaN,NaN,Wed Nov 11 02:17:00 +0000 2020,1326348167521193985,#DisneyPlus: O ator brasileiro Rodrigo Santoro...
